In [0]:
sh
from pyspark.sql import functions
device_code_dict = {1: {'1': 'Android-phone', '2': 'Android-tablet'}, 2: {'1': 'ios-phone', '2': 'ios-tablet'}}
raw_df = spark.read.parquet('s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=v3.0.0/range_type=DAY/date=2019-11-22/')
unified_df = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=daily/date=2019-11-22/')
print unified_df.count(), raw_df.count()
raw_df = raw_df.withColumn('device_code', functions.UserDefinedFunction(
        lambda x, y: device_code_dict[x][y])(raw_df['platform'], raw_df['device_type']))
df_join = raw_df.join(unified_df, on=[raw_df.app_id == unified_df.app_id, raw_df.country == unified_df.country_code, raw_df.device_code == unified_df.device_code], how='inner')
print df_join.count()

In [0]:

raw_df = spark.read.parquet('s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=v3.0.0/range_type=DAY/date=2019-11-23/')
unified_df = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=daily/date=2019-11-23/')
print unified_df.count(), raw_df.count()
print unified_df.select('app_id').distinct().count()
join_df = raw_df.join(unified_df, on='app_id',how='inner')
join_df.show()

In [0]:

df1 = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=daily/date=2019-10-31/')
df2 = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v2/fact/granularity=daily/date=2019-10-31/').select('app_id', 'country_code', 'device_code','est_install_base')
join_df = df1.join(df2, on=['app_id', 'country_code', 'device_code'], how='inner')
join_df.select('est_install_base', (join_df['est_install_penetration'] *
                            join_df['est_average_active_users'] / join_df['est_usage_penetration']).
                       alias('est_install_base_new')).na.fill(0).collect()

In [0]:

df1 = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=daily/date=2019-10-31/')
df2 = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v2/fact/granularity=daily/date=2019-10-31/')
df1.join(df2, on=['app_id', 'device_code', 'country_code'], how='inner').select('est_average_bytes_per_user', 'est_average_mb_per_user').show()

In [0]:

from pyspark.sql import Row
import unittest
from pyspark.sql.utils import AnalysisException


wrong_result = []
platform_list = {1: 'android', 2: 'ios'}
raw_granularity_dict = {'daily': 'DAY', 'monthly': 'MONTH', 'weekly': 'WEEK'}
device_code_dict = {1: {'1': 'Android-phone', '2': 'Android-tablet'}, 2: {'1': 'ios-phone', '2': 'ios-tablet'}}


def check_new_metrics_test_result(result_list, date, metric_name):
    for row in result_list:
        if row[0] != row[1]:
            wrong_result.append((metric_name, date[0], row[0], row[1]))
            print 'Test Wrong !!!! metric: {}, unified_v2: {}, unified_v1: {}, date: {}'.format(
                metric_name, row[0], row[1], date[0])


def get_path_date_list(granularity):
    df_date = spark.read.parquet(
        "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v2/fact/granularity=daily/").select('date').dropDuplicates()
    collect_date = df_date.collect()
    return collect_date


def check_usage_new_metrics(date_list, _granularity):
    unified_v1_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=daily/date={}/'
    unified_v2_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v2/fact/granularity=daily/date={}/'
    for date in date_list:
        unified_v1_path_parse = unified_v1_path.format(date[0])
        unified_v2_path_parse = unified_v2_path.format(date[0])
        unified_v1_df = spark.read.parquet(unified_v1_path_parse)
        unified_v2_df = spark.read.parquet(unified_v2_path_parse).select(
            'app_id', 'country_code', 'device_code', 'est_install_base', 'est_population'
            , 'est_total_sessions', 'est_total_time')
        join_df = unified_v1_df.join(unified_v2_df, on=['app_id', 'country_code', 'device_code'], how='inner')

        IB_result_row_list = join_df.select('est_install_base', (
                    join_df['est_install_penetration'] * join_df['est_average_active_users'] / join_df['est_usage_penetration']).
                       alias('est_install_base_new')).na.fill(0).collect()

        POP_result_row_list = join_df.select('est_population', (
                    join_df['est_average_active_users'] / join_df['est_usage_penetration']).
                       alias('est_population_new')).na.fill(0).collect()

        TS_result_row_list = join_df.select('est_total_sessions', (
                    join_df['est_average_active_users'] * join_df['est_average_session_per_user']).
                       alias('est_total_sessions_new')).na.fill(0).collect()

        TT_result_row_list = join_df.select('est_total_time', (
                    join_df['est_average_active_users'] * join_df['est_average_time_per_user'] / 60).
                       alias('est_total_sessions_new')).na.fill(0).collect()
        check_new_metrics_test_result(IB_result_row_list, date, 'est_install_base')
        check_new_metrics_test_result(POP_result_row_list, date, 'est_population')
        check_new_metrics_test_result(TS_result_row_list, date, 'est_total_sessions')
        check_new_metrics_test_result(TT_result_row_list, date, 'est_total_time')
        print 'date={} Test Complete !'.format(date[0])


graularity_list = ["daily"]
for graularity in graularity_list:
    check_usage_new_metrics(get_path_date_list(graularity), graularity)

In [0]:

raw_df = spark.read.parquet('s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=v3.0.0/range_type=DAY/date=2019-11-22/')
raw_df.printSchema()
raw_df.select('device_type').show(10)

In [0]:
%%sh
aws s3 ls 's3://b2c-prod-data-pipeline-qa/aa.store/result_store_unified_db_count_0430/'

In [0]:

df = spark.read.parquet('s3://b2c-prod-data-pipeline-qa/aa.store/result_store_unified_db_count_0430/')
df.show()

In [0]:

df = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v2/fact/granularity=daily/date=2015-12-28/')
df.filter('app_id=115 and country_code="JP" ').show()

In [0]:

df = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/granularity=daily/date=2015-12-28/')
df.show(3)

In [0]:
%%sh
aws s3 ls 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity=daily/month=2015-12/device_id=1001/store_id=1/'


In [0]:

df = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity=daily/month=2015-12/device_id=1001/store_id=1/')
df.printSchema()
df.show(5)
print df.filter("date = '2015-12-30'").count(), df.filter("date = '2015-12-30'").select('app_id', 'kpi').distinct().count()

In [0]:

df_date = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity=weekly/month=2013-01/").select('date').dropDuplicates()
collect_date = df_date.collect()
print collect_date

In [0]:
%%sh
aws s3 ls "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity=daily/month=2015-12/"


In [0]:

df_date = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity=weekly/month=2013-02/").select('date').dropDuplicates()
collect_date = df_date.collect()
print collect_date

In [0]:

from pyspark.sql.functions import count
raw_count_with_KPI = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity=daily/month=2015-12/device_id=1001/store_id=1/').filter("date='2015-12-27'").select("kpi", "app_id").distinct().groupBy("kpi").agg(count("kpi")).collect()
print raw_count_with_KPI

In [0]:
%%sh
aws s3 ls "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity=daily/month=2015-12/device_id=2002/store_id=0/"


In [0]:
 
df = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity=daily/month=2015-12/')
print df.filter(df.kpi==17).show()


In [0]:

df = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=daily/date=2015-12-31/')
print df.columns
df.select('est_average_bytes_per_session').filter(df.app_id==20600005138490).show(3)

In [0]:

import datetime
from dateutil.relativedelta import relativedelta
from pyspark.sql.utils import AnalysisException
from pyspark.sql.functions import count
from pyspark.sql import Row
from aadatapipelinecore.core.utils.retry import retry


test_result = []
kpi_mapping = {1: "est_average_active_users", 2: "est_average_session_per_user", 3: "est_average_session_duration",
               4: "est_install_penetration", 5: "est_average_active_days", 6: "est_percentage_active_days",
               7: "est_average_bytes_per_user", 8: "est_average_time_per_user", 9: "est_usage_penetration",
               10: "est_open_rate", 11: "est_total_time", 12: "est_share_of_category_time", 14: "est_total_sessions",
               15: "est_share_of_category_session", 17: "est_average_bytes_per_session",
               18: "est_share_of_category_bytes", 20: "est_percent_of_wifi_total", 21: "est_mb_per_second",
               22: "est_panel_size", 23: "est_installs", 24: "est_average_active_users_country_share",
               25: "est_installs_country_share", 26: "est_audience_index", 27: "est_audience_percentage",
               28: "est_cross_product_affinity"}


def write_test_result(df_write_result):
    df_write_result.write.format("delta").save(
        "s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_dump_unified_v1_daily_count_0511/",
        mode="append",
        partitionBy=["type"])


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 01, 31)
    start = datetime.date(2013, 01, 31)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 02, 15)
    start = datetime.date(2013, 01, 12)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 02, 15)
    start = datetime.date(2015, 12, 27)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(granularity):
    if granularity == 'daily':
        collect_date = get_daily_date_list()
    if granularity == 'weekly':
        collect_date = get_weekly_date_list()
    if granularity == 'monthly':
        collect_date = get_monthly_date_list()
    date_list = [(x[0][:7], x[0]) for x in collect_date]
    return date_list


def check_au_app_data_count(_granularity, date_list):
    for m in date_list:
        try:
            raw_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity={raw_granularity}/" \
                       "month={raw_month}/"
            unified_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/" \
                           "granularity={unified_granularity}/date={unified_date}/"
            raw_path_parse = raw_path.format(raw_month=m[0], raw_granularity=_granularity)
            raw_count_with_KPI = spark.read.parquet(raw_path_parse).filter(
                "date='{}'".format(m[1])).select("kpi", "app_id").groupBy(
                "kpi").agg(count("kpi")).collect()
            # print raw_count_with_KPI
        except AnalysisException as e:
            break
        for row in raw_count_with_KPI:
            unified_path_parse = unified_path.format(unified_date=m[1], unified_granularity=_granularity)
            unified_count = spark.read.parquet(unified_path_parse).filter(
                "{} is not null".format(kpi_mapping[row["kpi"]])).select(kpi_mapping[row["kpi"]]).count()
            if row["count(kpi)"] != unified_count:
                print 'Count Test Wrong !!!! raw data: {}, unified data: {}, date: {}, KPI {}'.format(
                                  row["count(kpi)"], unified_count, m[1], kpi_mapping[row["kpi"]])
            test_result.append((_granularity, m[1], row["count(kpi)"], unified_count, kpi_mapping[row["kpi"]]))
            print row["count(kpi)"], unified_count
        print "date={} test complete!".format(m[1])
        return


granularity_list = ["daily"]
for granularity in granularity_list:
    check_au_app_data_count(granularity, get_path_date_list(granularity))
print 'pass'
df_write_result = spark.createDataFrame(
    test_result, schema=['type', 'date', 'dump', 'unified_v1', 'kpi'])
write_test_result(df_write_result)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_dump_unified_v1_daily_count_0511/


In [0]:

df = spark.read.parquet('s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_dump_unified_v1_daily_count_0511/')
df.count()


In [0]:

aws s3 rm part-00001-fe64dcc4-6ec8-464b-bf6f-1ddabafb50f9-c000.snappy.parquet